In [1]:
import flopy.modflow as fm
import mfexport

## export a model

#### load the model using flopy

In [2]:
m = fm.Modflow.load('lpr_inset.nam',
                    model_ws='data/lpr')
m

/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/flopy/mbase.py:85: UserWarning: The program mf2005 does not exist or is not executable.
  warn(
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


MODFLOW 3 layer(s) 137 row(s) 171 column(s) 12 stress period(s)

#### define the location of the model grid
* spacing in meters

In [3]:
grid = mfexport.MFexportGrid(delr=m.dis.delr.array * .3048, # grid spacing in meters
                             delc=m.dis.delc.array * .3048, 
                             xul=557571, yul=446166, # upper left corner in CRS
                             epsg=3070 # epsg reference for projected CRS
                            )

/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/flopy/utils/crs.py:114: PendingDeprecationWarning: the prj argument will be deprecated and will be removed in version 3.6. Use prjfile instead.
  warnings.warn(
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/flopy/utils/crs.py:121: PendingDeprecationWarning: the epsg argument will be deprecated and will be removed in version 3.6. Use crs instead.
  warnings.warn(
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/flopy/utils/crs.py:140: PendingDeprecationWarning: the proj4 argument will be deprecated and will be removed in version 3.6. Use crs instead.
  warnings.warn(
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/flopy/utils/crs.py:121: PendingDeprecationWarning: the epsg argument will be deprecated and will be removed in version 3.6. Use crs instead.
  warnings.warn(
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/flopy/utils/cr

### export all data arrays and stress period data (`MfLists`) to shapefiles, rasters and PDFs
* array data are written to compressed GeoTIFFs
* transient lists are written to shapefiles
    * only model cells in the list are included
    * stress period data are included as attributes; only periods where stresses change are included
* all data are displayed in PDFs using `matplotlib.pyplot.imshow`

In [4]:
%%capture
mfexport.export(m, grid, output_path='postproc')

### export a package

In [5]:
%%capture
mfexport.export(m, grid, 'wel', output_path='postproc')

### summarize model input

In [6]:
df = mfexport.summarize(m)
df.head(20)

summarizing lpr_inset input...


,package,variable,layer,min,mean,max,period
0,DIS,botm,0.0,1033.500000,1049.306527,1066.699951,NaN
1,DIS,botm,1.0,991.239990,1004.092457,1063.699951,NaN
2,DIS,botm,2.0,988.239990,1000.740342,1042.599976,NaN
3,DIS,model_top,NaN,1071.079956,1105.837196,1190.459961,NaN
4,BAS6,ibound,0.0,1.000000,1.000000,1.000000,NaN
5,BAS6,ibound,1.0,1.000000,1.000000,1.000000,NaN
6,BAS6,ibound,2.0,1.000000,1.000000,1.000000,NaN
7,BAS6,strt,0.0,1069.800049,1087.256670,1107.099976,NaN
8,BAS6,strt,1.0,1069.900024,1087.247706,1107.099976,NaN
9,BAS6,strt,2.0,1070.000000,1087.247450,1107.099976,NaN


### Export model results

#### cell by cell flows
* written to compressed GeoTIFFs

In [7]:
mfexport.export_cell_budget('data/lpr/lpr_inset.cbc', 
                   grid,
                   kstpkper=(4, 0), output_path='postproc')

Exporting cell budget info...
file: data/lpr/lpr_inset.cbc
binary grid file: None
stress period 0, timestep 4
wrote postproc/rasters/STORAGE_per0_stp4.tif
wrote postproc/rasters/CONSTANT HEAD_per0_stp4.tif
wrote postproc/rasters/FLOW RIGHT FACE_per0_stp4.tif
wrote postproc/rasters/FLOW FRONT FACE_per0_stp4.tif
wrote postproc/rasters/FLOW LOWER FACE_per0_stp4.tif
wrote postproc/rasters/WELLS_per0_stp4.tif
wrote postproc/rasters/RECHARGE_per0_stp4.tif
wrote postproc/rasters/STREAM LEAKAGE_per0_stp4.tif


['postproc/rasters/STORAGE_per0_stp4.tif',
 'postproc/rasters/CONSTANT HEAD_per0_stp4.tif',
 'postproc/rasters/FLOW RIGHT FACE_per0_stp4.tif',
 'postproc/rasters/FLOW FRONT FACE_per0_stp4.tif',
 'postproc/rasters/FLOW LOWER FACE_per0_stp4.tif',
 'postproc/rasters/WELLS_per0_stp4.tif',
 'postproc/rasters/RECHARGE_per0_stp4.tif',
 'postproc/rasters/STREAM LEAKAGE_per0_stp4.tif']

#### heads
* compressed GeoTIFFs
* shapefiles of equipotential contours

In [8]:
headsfile = 'data/lpr/lpr_inset.hds'

mfexport.export_heads(headsfile, grid, hdry=m.upw.hdry, hnflo=m.bas6.hnoflo,
                      kstpkper=(4, 0),
                      output_path='postproc')

Exporting heads...
file: data/lpr/lpr_inset.hds
stress period 0, timestep 4
wrote postproc/rasters/wt_per0_stp4.tif
writing postproc/shps/wt_ctr_per0_stp4.shp... Done


/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/gisutils/shapefile.py:117: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(fiona.__gdal_version__) < LooseVersion("3.0.0"):
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/gisutils/shapefile.py:117: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(fiona.__gdal_version__) < LooseVersion("3.0.0"):
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/gisutils/shapefile.py:129: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  Type = g.type


['postproc/rasters/wt_per0_stp4.tif', 'postproc/shps/wt_ctr_per0_stp4.shp']

#### drawdown
* compressed GeoTIFFs
* shapefiles of equipotential contours

In [9]:
mfexport.export_drawdown(headsfile, grid, hdry=m.upw.hdry, hnflo=m.bas6.hnoflo,
                         kstpkper0=(4, 0),
                         kstpkper1=(4, 8),
                         output_path='postproc')

Exporting drawdown...
file: data/lpr/lpr_inset.hds
from stress period 0, timestep 4
to stress period 8, timestep 4


wrote postproc/rasters/wt-ddn_per8_stp4.tif
writing postproc/shps/wt-ddn_ctr_per8_stp4.shp... Done


/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/gisutils/shapefile.py:117: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(fiona.__gdal_version__) < LooseVersion("3.0.0"):
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/gisutils/shapefile.py:117: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(fiona.__gdal_version__) < LooseVersion("3.0.0"):
/home/runner/micromamba/envs/mfexport_ci/lib/python3.11/site-packages/gisutils/shapefile.py:129: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  Type = g.type


['postproc/rasters/wt-ddn_per8_stp4.tif',
 'postproc/shps/wt-ddn_ctr_per8_stp4.shp']

#### SFR results
* shapefile of SFR water balance results
* PDFs of simulated stream flows and stream/aquifer interactions

In [10]:
%%capture
mf2005_sfr_outputfile = 'data/lpr/lpr_inset.sfr.out'
outfiles = mfexport.export_sfr_results(mf2005_sfr_outputfile=mf2005_sfr_outputfile,
                                       model=m,
                                       grid=grid,
                                       kstpkper=(4, 0),
                                       output_length_units='feet',
                                       output_time_units='seconds',
                                       output_path='postproc'
                                       )